# 多进程
### 要让Python程序实现多进程（multiprocessing），我们先了解操作系统的相关知识。

### Unix/Linux操作系统提供了一个fork()系统调用，它非常特殊。普通的函数调用，调用一次，返回一次，但是fork()调用一次，返回两次，因为操作系统自动把当前进程（称为父进程）复制了一份（称为子进程），然后，分别在父进程和子进程内返回。

### 子进程永远返回0，而父进程返回子进程的ID。这样做的理由是，一个父进程可以fork出很多子进程，所以，父进程要记下每个子进程的ID，而子进程只需要调用getppid()就可以拿到父进程的ID。

In [11]:
# Python的os模块封装了常见的系统调用，其中就包括fork，可以在Python程序中轻松创建子进程：
import os

print('Process (%s) start...' % os.getpid()) # 只在Unix/Linux/Mac可用
pid  = os.fork() # 这个方法会返回两次
print(pid)
if pid != 0:
    print("父进程%s创建了一个子进程%s" % (os.getpid(),pid))
else:
    print("子进程：%s;父进程：%s" % (os.getpid(),os.getppid()))
    
# 由于Windows没有fork调用，上面的代码在Windows上无法运行。
# 而Mac系统是基于BSD（Unix的一种）内核，所以，在Mac下运行是没有问题的，推荐大家用Mac学Python！

# 有了fork调用，一个进程在接到新任务时就可以复制出一个子进程来处理新任务，
# 常见的Apache服务器就是由父进程监听端口，每当有新的http请求时，就fork出子进程来处理新的http请求。

Process (4108) start...
4717
父进程4108创建了一个子进程4717
0
子进程：4717;父进程：4108


In [14]:
# multiprocessing
# 如果你打算编写多进程的服务程序，Unix/Linux无疑是正确的选择。
# 由于Windows没有fork调用，难道在Windows上无法用Python编写多进程的程序？

# 由于Python是跨平台的，自然也应该提供一个跨平台的多进程支持。multiprocessing模块就是跨平台版本的多进程模块。

# multiprocessing模块提供了一个Process类来代表一个进程对象，下面的例子演示了启动一个子进程并等待其结束：
from multiprocessing import Process
import os
import time

def run_proc(name):
    time.sleep(3) # 延迟三秒执行
    print('子进程 %s (%s)运行中...' % (name,os.getpid()))
    
print('父进程：%s' % os.getpid())
p = Process(target=run_proc,args=('test',))
print('子进程即将启动')
p.start()
p.join() # join()方法可以等待子进程结束后再继续往下运行，通常用于进程间的同步。
print('子进程结束')
# 创建子进程时，只需要传入一个执行函数和函数的参数，
# 创建一个Process实例，用start()方法启动，这样创建进程比fork()还要简单。

父进程：4108
子进程即将启动
子进程 test (4760)运行中...
子进程结束


# 进程池 Pool

In [15]:
# 如果要启动大量的子进程，可以用进程池的方式批量创建子进程：
from multiprocessing import Pool
import os,time,random

def long_time_task(name):
    print("任务%s(%s)开始执行..." % (name,os.getpid()))
    start_time = time.time()
    time.sleep(random.random() * 3)
    end_time = time.time()
    print("任务%s执行完毕，总共耗时%s" % (name,(end_time - start_time)))

print('父进程：%s' % os.getpid())
p = Pool(4)
for i in range(5): # 因为线程池大小为4，所有第五个任务4需要等有一个任务执行完毕后才能执行
    p.apply_async(long_time_task,args=(i,))
print('等待所有子任务执行完毕...')
p.close() # 调用join()前必须先调用close(),调用close()之后就不能继续添加新的Process了
p.join() # 对Pool对象调用join()方法会等待所有子进程执行完毕
print("所有子任务执行完毕")

# 由于Pool的默认大小是CPU的核数，如果你不幸拥有8核CPU，你要提交至少9个子进程才能看到上面的等待效果。

父进程：4108
任务0(4795)开始执行...
任务1(4796)开始执行...
任务2(4797)开始执行...
任务3(4798)开始执行...
任务1执行完毕，总共耗时0.0596158504486084
任务4(4796)开始执行...
等待所有子任务执行完毕...
任务2执行完毕，总共耗时0.4725229740142822
任务3执行完毕，总共耗时0.47954416275024414
任务0执行完毕，总共耗时0.5764222145080566
任务4执行完毕，总共耗时0.6994340419769287
所有子任务执行完毕


In [17]:
# 子进程
# 很多时候，子进程并不是自身，而是一个外部进程。我们创建了子进程后，还需要控制子进程的输入和输出。

# subprocess模块可以让我们非常方便地启动一个子进程，然后控制其输入和输出。

# 下面的例子演示了如何在Python代码中运行命令nslookup www.python.org，这和命令行直接运行的效果是一样的
import subprocess

print('$ nslookup www.python.org')
r = subprocess.call(['nslookup','www.python.org'])
# 执行结果如果下，不过在jupyter 中无法显示
# Server:		10.237.28.10
# Address:	10.237.28.10#53

# Non-authoritative answer:
# www.python.org	canonical name = dualstack.python.map.fastly.net.
# Name:	dualstack.python.map.fastly.net
# Address: 151.101.76.223
print("Exit code:",r)

$ nslookup www.python.org


0

In [18]:
# 如果子进程还需要输入，则可以通过communicate()方法输入：
import subprocess

print('$ nslookup')
p = subprocess.Popen(['nslookup'],stdin=subprocess.PIPE,stdout=subprocess.PIPE,
                     stderr=subprocess.PIPE)
output,err = p.communicate(b'set q=mx\npython.org\nexit\n')
print(output.decode('utf-8'))
print('Exit code:',p.returncode)
# 上面的代码相当于在命令行执行命令nslookup，然后手动输入：
# set q=mx
# python.org
# exit

$ nslookup
Server:		10.237.28.10
Address:	10.237.28.10#53

Non-authoritative answer:
python.org	mail exchanger = 50 mail.python.org.

Authoritative answers can be found from:


Exit code: 0


In [22]:
# 进程间通信
# Process之间肯定是需要通信的，操作系统提供了很多机制来实现进程间的通信。
# Python的multiprocessing模块包装了底层的机制，提供了Queue、Pipes等多种方式来交换数据。

# 我们以Queue为例，在父进程中创建两个子进程，一个往Queue里写数据，一个从Queue里读数据：
from multiprocessing import Process, Queue
import os,time,random

# 写数据进程执行的方法
def write(q):
    print('进程%s：开始写数据' % os.getpid())
    for value in ['A','B','C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())

# 读数据进行执行的方法
def read(q):
    print('进程%s：开始读取数据'% os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)

q = Queue() # 父进程创建一个队列
pw = Process(target=write,args=(q,)) # 写进程
pr = Process(target=read,args=(q,)) # 读进程
# 启动写进程pw，写入:
pw.start()

# 启动读进程，读出：
pr.start()

# 等待pw进程结束
pw.join()

# pr进程是死循环，无法等待期结束，只能强行终止:
pr.terminate()

print("两个进程都执行完毕!")

进程5210：开始写数据
Put A to queue...
进程5211：开始读取数据
Get A from queue.
Put B to queue...
Get B from queue.
Put C to queue...
Get C from queue.
两个进程都执行完毕!


# 多进程小结
#### 在Unix/Linux下，multiprocessing模块封装了fork()调用，使我们不需要关注fork()的细节。由于Windows没有fork调用，因此，multiprocessing需要“模拟”出fork的效果，父进程所有Python对象都必须通过pickle序列化再传到子进程去，所以，如果multiprocessing在Windows下调用失败了，要先考虑是不是pickle失败了。


#### 在Unix/Linux下，可以使用fork()调用实现多进程。

##### 要实现跨平台的多进程，可以使用multiprocessing模块。

#### 进程间通信是通过Queue、Pipes等实现的。

# 多线程
#### 多任务可以由多进程完成，也可以由一个进程内的多线程完成。

#### 我们前面提到了进程是由若干线程组成的，一个进程至少有一个线程。

#### 由于线程是操作系统直接支持的执行单元，因此，高级语言通常都内置多线程的支持，Python也不例外，并且，Python的线程是真正的Posix Thread，而不是模拟出来的线程。

#### Python的标准库提供了两个模块：_thread和threading，_thread是低级模块，threading是高级模块，对_thread进行了封装。绝大多数情况下，我们只需要使用threading这个高级模块。

In [23]:
# 启动一个线程就是把一个函数传入并创建Thread实例，然后调用start()开始执行：
import time,threading

# 定义线程执行的代码:
def loop():
    print("线程 %s 执行中..." % threading.current_thread().name)
    n = 0
    while n < 5:
        n += 1
        print('线程 %s >>> %s' % (threading.current_thread().name,n))
        time.sleep(1)
    print('线程 %s 执行完毕！' % threading.current_thread().name)

print('线程 %s 开始执行...' % threading.current_thread().name)
t = threading.Thread(target=loop,name='LoopThread')
t.start()
t.join()
print('线程 %s 执行结束.' % threading.current_thread().name)
# 由于任何进程默认就会启动一个线程，我们把该线程称为主线程，
# 主线程又可以启动新的线程，Python的threading模块有个current_thread()函数，它永远返回当前线程的实例。
# 主线程实例的名字叫MainThread，子线程的名字在创建时指定，我们用LoopThread命名子线程。
# 名字仅仅在打印时用来显示，完全没有其他意义，如果不起名字Python就自动给线程命名为Thread-1，Thread-2……

线程 MainThread 开始执行...
线程 LoopThread 执行中...
线程 LoopThread >>> 1
线程 LoopThread >>> 2
线程 LoopThread >>> 3
线程 LoopThread >>> 4
线程 LoopThread >>> 5
线程 LoopThread 执行完毕！
线程 MainThread 执行结束.


In [26]:
# Lock 线程锁机制
# 多线程和多进程最大的不同在于，多进程中，同一个变量，各自有一份拷贝存在于每个进程中，互不影响，
# 而多线程中，所有变量都由所有线程共享，所以，任何一个变量都可以被任何一个线程修改，
# 因此，线程之间共享数据最大的危险在于多个线程同时改一个变量，把内容给改乱了。

# 来看看多个线程同时操作一个变量怎么把内容给改乱了：

balance = 0 # 定义一个银行存款变量

def change_it(n):
    # 先存后取，结果应该为0：
    global balance
    balance = balance + n
    balance = balance - n
    
def run_thread(n):
    for i in range(1000000): # 这个必须要大一些，不然不好复现
        change_it(n)

t1 = threading.Thread(target=run_thread,args=(5,))
t2 = threading.Thread(target=run_thread,args=(8,))
t1.start()
t2.start()
t1.join()
t2.join()
print("最终的存款为:",balance) 
# 我们定义了一个共享变量balance，初始值为0，并且启动两个线程，先存后取，理论上结果应该为0，
# 但是，由于线程的调度是由操作系统决定的，当t1、t2交替执行时，只要循环次数足够多，balance的结果就不一定是0了。

最终的存款为: 24


#### 原因是因为高级语言的一条语句在CPU执行时是若干条语句，即使一个简单的计算：
```python
balance = balance + n
```
#### 也分两步：

#### 计算balance + n，存入临时变量中；
#### 将临时变量的值赋给balance。
#### 也就是可以看成：
```python
x = balance + n
balance = x
```
#### 由于x是局部变量，两个线程各自都有自己的x，当代码正常执行时：
```python
初始值 balance = 0

t1: x1 = balance + 5 # x1 = 0 + 5 = 5
t1: balance = x1     # balance = 5
t1: x1 = balance - 5 # x1 = 5 - 5 = 0
t1: balance = x1     # balance = 0

t2: x2 = balance + 8 # x2 = 0 + 8 = 8
t2: balance = x2     # balance = 8
t2: x2 = balance - 8 # x2 = 8 - 8 = 0
t2: balance = x2     # balance = 0
    
结果 balance = 0
```
#### 但是t1和t2是交替运行的，如果操作系统以下面的顺序执行t1、t2：
```python
初始值 balance = 0

t1: x1 = balance + 5  # x1 = 0 + 5 = 5

t2: x2 = balance + 8  # x2 = 0 + 8 = 8
t2: balance = x2      # balance = 8

t1: balance = x1      # balance = 5
t1: x1 = balance - 5  # x1 = 5 - 5 = 0
t1: balance = x1      # balance = 0

t2: x2 = balance - 8  # x2 = 0 - 8 = -8
t2: balance = x2   # balance = -8

结果 balance = -8
```
#### 究其原因，是因为修改balance需要多条语句，而执行这几条语句时，线程可能中断，从而导致多个线程把同一个对象的内容改乱了。

#### 两个线程同时一存一取，就可能导致余额不对，你肯定不希望你的银行存款莫名其妙地变成了负数，所以，我们必须确保一个线程在修改balance的时候，别的线程一定不能改。

#### 如果我们要确保balance计算正确，就要给change_it()上一把锁，当某个线程开始执行change_it()时，我们说，该线程因为获得了锁，因此其他线程不能同时执行change_it()，只能等待，直到锁被释放后，获得该锁以后才能改。由于锁只有一个，无论多少线程，同一时刻最多只有一个线程持有该锁，所以，不会造成修改的冲突。创建一个锁就是通过threading.Lock()来实现：

In [31]:
# 使用lock机制来避免冲突问题
import time,threading
balance2 = 0
lock = threading.Lock()

def change_it(n):
    global balance2
    balance2 = balance2 + n
    balance2 = balance2 - n

def run_thread_with_lock(n):
    for i in range(1000000):
        # 先要获取锁
        lock.acquire()
        try:
            # 现在可以放心的修改了
            change_it(n)
        finally:
            # 最后要无论是操作完还会发生异常都要将锁释放
            lock.release()
tl1 = threading.Thread(target=run_thread_with_lock,args=(5,))
tl2 = threading.Thread(target=run_thread_with_lock,args=(8,))
tl1.start()
tl2.start()
tl1.join()
tl2.join()
print("最终操作完后的存款为：",balance2)

# 当多个线程同时执行lock.acquire()时，只有一个线程能成功地获取锁，然后继续执行代码，其他线程就继续等待直到获得锁为止。

# 获得锁的线程用完后一定要释放锁，否则那些苦苦等待锁的线程将永远等待下去，成为死线程。
# 所以我们用try...finally来确保锁一定会被释放。

# 锁的好处就是确保了某段关键代码只能由一个线程从头到尾完整地执行，
# 坏处当然也很多，首先是阻止了多线程并发执行，包含锁的某段代码实际上只能以单线程模式执行，效率就大大地下降了。
# 其次，由于可以存在多个锁，不同的线程持有不同的锁，并试图获取对方持有的锁时，可能会造成死锁，
# 导致多个线程全部挂起，既不能执行，也无法结束，只能靠操作系统强制终止。

最终操作完后的存款为： 0


# 多核CPU
#### 如果你不幸拥有一个多核CPU，你肯定在想，多核应该可以同时执行多个线程。

#### 如果写一个死循环的话，会出现什么情况呢？

#### 打开Mac OS X的Activity Monitor，或者Windows的Task Manager，都可以监控某个进程的CPU使用率。

#### 我们可以监控到一个死循环线程会100%占用一个CPU。

#### 如果有两个死循环线程，在多核CPU中，可以监控到会占用200%的CPU，也就是占用两个CPU核心。

#### 要想把N核CPU的核心全部跑满，就必须启动N个死循环线程。

In [33]:
# 试试用Python写个死循环：
import threading,multiprocessing

# 写一个死循环
def loop_forever():
    x = 0
    while True:
        x = x ^ 1
print('当前电脑的CPU数目为：',multiprocessing.cpu_count())
for i in range(multiprocessing.cpu_count()):
    t = threading.Thread(target=loop_forever)
    t.start()
# 启动与CPU核心数量相同的N个线程，在4核CPU上可以监控到CPU占用率仅有102%，也就是仅使用了一核。

# 但是用C、C++或Java来改写相同的死循环，直接可以把全部核心跑满，4核就跑到400%，8核就跑到800%，
# 为什么Python不行呢？

# 因为Python的线程虽然是真正的线程，但解释器执行代码时，有一个GIL锁：Global Interpreter Lock，
# 任何Python线程执行前，必须先获得GIL锁，然后，每执行100条字节码，解释器就自动释放GIL锁，让别的线程有机会执行。这个GIL全局锁实际上把所有线程的执行代码都给上了锁，所以，多线程在Python中只能交替执行，即使100个线程跑在100核CPU上，也只能用到1个核。

# GIL是Python解释器设计的历史遗留问题，通常我们用的解释器是官方实现的CPython，
# 要真正利用多核，除非重写一个不带GIL的解释器。

# 所以，在Python中，可以使用多线程，但不要指望能有效利用多核。
# 如果一定要通过多线程利用多核，那只能通过C扩展来实现，不过这样就失去了Python简单易用的特点。

# 不过，也不用过于担心，Python虽然不能利用多线程实现多核任务，但可以通过多进程实现多核任务。
# 多个Python进程有各自独立的GIL锁，互不影响。

当前电脑的CPU数目为： 16


# 多线程小结
#### 多线程编程，模型复杂，容易发生冲突，必须用锁加以隔离，同时，又要小心死锁的发生。

#### Python解释器由于设计时有GIL全局锁，导致了多线程无法利用多核。多线程的并发在Python中就是一个美丽的梦。

# ThreadLocal

#### 在多线程环境下，每个线程都有自己的数据。一个线程使用自己的局部变量比使用全局变量好，因为局部变量只有线程自己能看见，不会影响其他线程，而全局变量的修改必须加锁。

#### 但是局部变量也有问题，就是在函数调用的时候，传递起来很麻烦：
```python
def process_student(name):
    std = Student(name)
    # std是局部变量，但是每个函数都要用它，因此必须传进去：
    do_task_1(std)
    do_task_2(std)

def do_task_1(std):
    do_subtask_1(std)
    do_subtask_2(std)

def do_task_2(std):
    do_subtask_2(std)
    do_subtask_2(std)
```
#### 每个函数一层一层调用都这么传参数那还得了？用全局变量？也不行，因为每个线程处理不同的Student对象，不能共享。

#### 如果用一个全局dict存放所有的Student对象，然后以thread自身作为key获得线程对应的Student对象如何？
```python
global_dict = {}

def std_thread(name):
    std = Student(name)
    # 把std放到全局变量global_dict中：
    global_dict[threading.current_thread()] = std
    do_task_1()
    do_task_2()

def do_task_1():
    # 不传入std，而是根据当前线程查找：
    std = global_dict[threading.current_thread()]
    ...

def do_task_2():
    # 任何函数都可以查找出当前线程的std变量：
    std = global_dict[threading.current_thread()]
    ...
```
#### 这种方式理论上是可行的，它最大的优点是消除了std对象在每层函数中的传递问题，但是，每个函数获取std的代码有点丑。
#### 有没有更简单的方式？

In [3]:
# ThreadLocal应运而生，不用查找dict，ThreadLocal帮你自动做这件事：
import threading

# 创建全局ThreadLocal对象
local_school = threading.local()

def process_student():
    # 取出当前线程关联的student
    student = local_school.student
    print("当前线程 %s 的学生是 %s" % (threading.current_thread().name,student))

def process_thread(name):
    # 绑定threadLocal的student
    local_school.student = name
    process_student()

t1 = threading.Thread(target=process_thread,args=("张三",),name="Thread-1")
t2 = threading.Thread(target=process_thread,args=("李四",),name="Thread-2")
t1.start()
t2.start()
t1.join()
t2.join()
print("所有任务执行完毕")
# 全局变量local_school就是一个ThreadLocal对象，每个Thread对它都可以读写student属性，但互不影响。
# 你可以把local_school看成全局变量，但每个属性如local_school.student都是线程的局部变量，可以任意读写而互不干扰，
# 也不用管理锁的问题，ThreadLocal内部会处理。

# 可以理解为全局变量local_school是一个dict，不但可以用local_school.student，还可以绑定其他变量，如local_school.teacher等等。

# ThreadLocal最常用的地方就是为每个线程绑定一个数据库连接，HTTP请求，用户身份信息等，
# 这样一个线程的所有调用到的处理函数都可以非常方便地访问这些资源。

# 小结
# 一个ThreadLocal变量虽然是全局变量，但每个线程都只能读写自己线程的独立副本，互不干扰。
# ThreadLocal解决了参数在一个线程中各个函数之间互相传递的问题。

当前线程 Thread-1 的学生是 张三
当前线程 Thread-2 的学生是 李四
所有任务执行完毕


# 进程 vs. 线程
#### 我们介绍了多进程和多线程，这是实现多任务最常用的两种方式。现在，我们来讨论一下这两种方式的优缺点。

#### 首先，要实现多任务，通常我们会设计Master-Worker模式，Master负责分配任务，Worker负责执行任务，因此，多任务环境下，通常是一个Master，多个Worker。

#### 如果用多进程实现Master-Worker，主进程就是Master，其他进程就是Worker。

#### 如果用多线程实现Master-Worker，主线程就是Master，其他线程就是Worker。

#### 多进程模式最大的优点就是稳定性高，因为一个子进程崩溃了，不会影响主进程和其他子进程。（当然主进程挂了所有进程就全挂了，但是Master进程只负责分配任务，挂掉的概率低）著名的Apache最早就是采用多进程模式。

#### 多进程模式的缺点是创建进程的代价大，在Unix/Linux系统下，用fork调用还行，在Windows下创建进程开销巨大。另外，操作系统能同时运行的进程数也是有限的，在内存和CPU的限制下，如果有几千个进程同时运行，操作系统连调度都会成问题。

#### 多线程模式通常比多进程快一点，但是也快不到哪去，而且，多线程模式致命的缺点就是任何一个线程挂掉都可能直接造成整个进程崩溃，因为所有线程共享进程的内存。在Windows上，如果一个线程执行的代码出了问题，你经常可以看到这样的提示：“该程序执行了非法操作，即将关闭”，其实往往是某个线程出了问题，但是操作系统会强制结束整个进程。

#### 在Windows下，多线程的效率比多进程要高，所以微软的IIS服务器默认采用多线程模式。由于多线程存在稳定性的问题，IIS的稳定性就不如Apache。为了缓解这个问题，IIS和Apache现在又有多进程+多线程的混合模式，真是把问题越搞越复杂。

# 线程切换
#### 无论是多进程还是多线程，只要数量一多，效率肯定上不去，为什么呢？

#### 我们打个比方，假设你不幸正在准备中考，每天晚上需要做语文、数学、英语、物理、化学这5科的作业，每项作业耗时1小时。

#### 如果你先花1小时做语文作业，做完了，再花1小时做数学作业，这样，依次全部做完，一共花5小时，这种方式称为单任务模型，或者批处理任务模型。

#### 假设你打算切换到多任务模型，可以先做1分钟语文，再切换到数学作业，做1分钟，再切换到英语，以此类推，只要切换速度足够快，这种方式就和单核CPU执行多任务是一样的了，以幼儿园小朋友的眼光来看，你就正在同时写5科作业。

#### 但是，切换作业是有代价的，比如从语文切到数学，要先收拾桌子上的语文书本、钢笔（这叫保存现场），然后，打开数学课本、找出圆规直尺（这叫准备新环境），才能开始做数学作业。操作系统在切换进程或者线程时也是一样的，它需要先保存当前执行的现场环境（CPU寄存器状态、内存页等），然后，把新任务的执行环境准备好（恢复上次的寄存器状态，切换内存页等），才能开始执行。这个切换过程虽然很快，但是也需要耗费时间。如果有几千个任务同时进行，操作系统可能就主要忙着切换任务，根本没有多少时间去执行任务了，这种情况最常见的就是硬盘狂响，点窗口无反应，系统处于假死状态。

#### 所以，多任务一旦多到一个限度，就会消耗掉系统所有的资源，结果效率急剧下降，所有任务都做不好。

# 计算密集型 vs. IO密集型
#### 是否采用多任务的第二个考虑是任务的类型。我们可以把任务分为计算密集型和IO密集型。

#### 计算密集型任务的特点是要进行大量的计算，消耗CPU资源，比如计算圆周率、对视频进行高清解码等等，全靠CPU的运算能力。这种计算密集型任务虽然也可以用多任务完成，但是任务越多，花在任务切换的时间就越多，CPU执行任务的效率就越低，所以，要最高效地利用CPU，计算密集型任务同时进行的数量应当等于CPU的核心数。

#### 计算密集型任务由于主要消耗CPU资源，因此，代码运行效率至关重要。Python这样的脚本语言运行效率很低，完全不适合计算密集型任务。对于计算密集型任务，最好用C语言编写。

#### 第二种任务的类型是IO密集型，涉及到网络、磁盘IO的任务都是IO密集型任务，这类任务的特点是CPU消耗很少，任务的大部分时间都在等待IO操作完成（因为IO的速度远远低于CPU和内存的速度）。对于IO密集型任务，任务越多，CPU效率越高，但也有一个限度。常见的大部分任务都是IO密集型任务，比如Web应用。

#### IO密集型任务执行期间，99%的时间都花在IO上，花在CPU上的时间很少，因此，用运行速度极快的C语言替换用Python这样运行速度极低的脚本语言，完全无法提升运行效率。对于IO密集型任务，最合适的语言就是开发效率最高（代码量最少）的语言，脚本语言是首选，C语言最差。

# 异步IO
#### 考虑到CPU和IO之间巨大的速度差异，一个任务在执行的过程中大部分时间都在等待IO操作，单进程单线程模型会导致别的任务无法并行执行，因此，我们才需要多进程模型或者多线程模型来支持多任务并发执行。

#### 现代操作系统对IO操作已经做了巨大的改进，最大的特点就是支持异步IO。如果充分利用操作系统提供的异步IO支持，就可以用单进程单线程模型来执行多任务，这种全新的模型称为事件驱动模型，Nginx就是支持异步IO的Web服务器，它在单核CPU上采用单进程模型就可以高效地支持多任务。在多核CPU上，可以运行多个进程（数量与CPU核心数相同），充分利用多核CPU。由于系统总的进程数量十分有限，因此操作系统调度非常高效。用异步IO编程模型来实现多任务是一个主要的趋势。

#### 对应到Python语言，单线程的异步编程模型称为协程，有了协程的支持，就可以基于事件驱动编写高效的多任务程序。我们会在后面讨论如何编写协程。

# 分布式进程
#### 在Thread和Process中，应当优选Process，因为Process更稳定，而且，Process可以分布到多台机器上，而Thread最多只能分布到同一台机器的多个CPU上。

#### Python的multiprocessing模块不但支持多进程，其中managers子模块还支持把多进程分布到多台机器上。一个服务进程可以作为调度者，将任务分布到其他多个进程中，依靠网络通信。由于managers模块封装很好，不必了解网络通信的细节，就可以很容易地编写分布式多进程程序。

#### 举个例子：如果我们已经有一个通过Queue通信的多进程程序在同一台机器上运行，现在，由于处理任务的进程任务繁重，希望把发送任务的进程和处理任务的进程分布到两台机器上。怎么用分布式进程实现？

#### 原有的Queue可以继续使用，但是，通过managers模块把Queue通过网络暴露出去，就可以让其他机器的进程访问Queue了。

#### 我们先看服务进程，服务进程负责启动Queue，把Queue注册到网络上，然后往Queue里面写入任务：

In [16]:
# task_master.py

import random,time,queue
from multiprocessing.managers import BaseManager

# 发送任务的队列
task_queue = queue.Queue()
# 接收任务的队列
result_queue = queue.Queue()

# 从BaseManager继承的QueueManager
class QueueManager(BaseManager):
    pass

# 把两个Queue都注册到网络上, callable参数关联了Queue对象:
QueueManager.register('get_task_queue',callable=lambda: task_queue)
QueueManager.register('get_result_queue',callable=lambda: result_queue)
# 绑定端口5000,设置验证码'abc'
manager = QueueManager(address=('',5000),authkey=b'abc')
# 启动Queue
manager.start()
try:
    # 获得通过网络访问的Queue对象
    task = manager.get_task_queue()
    result = manager.get_result_queue()
    # 放几个任务进去：
    for i in range(10):
        n = random.randint(0,10000)
        print('放入任务：%d' % n)
        task.put(n)
    # 从result队列读取结果:
    print('尝试获取结果...')
    for i in range(10):
        r = result.get(timeout=10)
        print('结果为：%s' % r)
finally:
    # 关闭
    manager.shutdown()
    print("manager 关闭")
# 请注意，当我们在一台机器上写多进程程序时，创建的Queue可以直接拿来用，
# 但是，在分布式多进程环境下，添加任务到Queue不可以直接对原始的task_queue进行操作，
# 那样就绕过了QueueManager的封装，必须通过manager.get_task_queue()获得的Queue接口添加。

放入任务：1112
放入任务：3474
放入任务：7582
放入任务：6742
放入任务：3123
放入任务：4102
放入任务：3960
放入任务：1252
放入任务：4769
放入任务：9829
尝试获取结果...
结果为：1112 * 1112 = 1236544
结果为：3474 * 3474 = 12068676
结果为：7582 * 7582 = 57486724
结果为：6742 * 6742 = 45454564
结果为：3123 * 3123 = 9753129
结果为：4102 * 4102 = 16826404
结果为：3960 * 3960 = 15681600
结果为：1252 * 1252 = 1567504
结果为：4769 * 4769 = 22743361
结果为：9829 * 9829 = 96609241
manager 关闭


In [ ]:
# 下面的代码放在’04-分布式的client‘的文件上执行的
# task_worker.py
# 然后，在另一台机器上启动任务进程（本机上启动也可以）：
import time,sys,queue
from multiprocessing.managers import BaseManager

# 创建类似的QueueManager
class QueueManager(BaseManager):
    pass

# 由于这个QueueManager只从网络上获取Queue，所以注册时只提供名字：
QueueManager.register('get_task_queue')
QueueManager.register('get_result_queue')

# 连接到服务器，也就是运行task_master.py的机器：
server_addr = '127.0.0.1'
print('连接服务器 %s...' % server_addr)
# 端口和验证码注意保持与task_master.py设置的完全一致：
m = QueueManager(address=(server_addr,5000),authkey=b'abc')
# 网络连接：
m.connect()
# 获取Queue的对象：
task_clent = m.get_task_queue()
result_clent = m.get_result_queue()
# 从task队列取出任务，并把结果写入result队列：
for i in range(10):
    try:
        n = task_clent.get(timeout=1)
        print('执行任务：%d * %d' % (n,n))
        r = '%d * %d = %d' % (n,n,n*n)
        time.sleep(1)
        result_clent.put(r)
    except queue.Empty:
        print("任务Queue为空！")
# 处理结束：
print("client任务结束")

#### 这个简单的Master/Worker模型有什么用？其实这就是一个简单但真正的分布式计算，把代码稍加改造，启动多个worker，就可以把任务分布到几台甚至几十台机器上，比如把计算n*n的代码换成发送邮件，就实现了邮件队列的异步发送。

#### Queue对象存储在哪？注意到task_worker.py中根本没有创建Queue的代码，所以，Queue对象存储在task_master.py进程中：

#### 而Queue之所以能通过网络访问，就是通过QueueManager实现的。由于QueueManager管理的不止一个Queue，所以，要给每个Queue的网络调用接口起个名字，比如get_task_queue。

#### authkey有什么用？这是为了保证两台机器正常通信，不被其他机器恶意干扰。如果task_worker.py的authkey和task_master.py的authkey不一致，肯定连接不上。

# 小结
#### Python的分布式进程接口简单，封装良好，适合需要把繁重任务分布到多台机器的环境下。

#### 注意Queue的作用是用来传递任务和接收结果，每个任务的描述数据量要尽量小。比如发送一个处理日志文件的任务，就不要发送几百兆的日志文件本身，而是发送日志文件存放的完整路径，由Worker进程再去共享的磁盘上读取文件。